In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
from benchmark.random_problem import RandomProblem

# Read in 100 randomly generated CNF(12, 4, r_4) instances
num_instances = 100
n = 12
k = 4
ps = [4, 8, 16]

rp = RandomProblem(type='ksat')
instances = rp.from_poisson(n=n, k=k, instances=num_instances, from_file=0, calc_naive=True)

In [3]:
import torch
import h5py
from k_sat.pytorch_solver.pytorch_circuit import PytorchCircuit
from k_sat.pytorch_solver.pytorch_optimiser import PytorchOptimiser

optimal_params = {}

for p in ps:
	print(f'Training for p = {p}')
	circuit = PytorchCircuit(num_vars=n, layers=p)
	adam = torch.optim.Adam(circuit.parameters(), lr=0.1, maximize=True)
	epochs = 60 if p < 5 else 80
	optimiser = PytorchOptimiser(circuit, adam, epochs=epochs)
	optimiser.find_optimal_params(instances.formulas)
	# Save to file
	parent_dir = os.path.dirname(os.getcwd())
	dir = f"{parent_dir}/benchmark/instances/n_{n}"
	with h5py.File(f'{dir}/a_params_{p}.hdf5', 'w') as file:
		file.create_dataset(f'gamma', data=circuit.gamma.detach().clone())
		file.create_dataset(f'beta', data=circuit.beta.detach().clone())

Training for p = 4
Epoch 0, p_succ: 0.0010931991273537278
Epoch 10, p_succ: 0.1500437706708908
Epoch 20, p_succ: 0.1633884757757187
Epoch 30, p_succ: 0.16702686250209808
Epoch 40, p_succ: 0.16732850670814514
Epoch 50, p_succ: 0.1680927723646164
Epoch 60, p_succ: 0.16813427209854126
Training for p = 8
Epoch 0, p_succ: 0.0012440548744052649
Epoch 10, p_succ: 0.3499329090118408
Epoch 20, p_succ: 0.36637580394744873
Epoch 30, p_succ: 0.3727189004421234
Epoch 40, p_succ: 0.3786788880825043
Epoch 50, p_succ: 0.38084280490875244
Epoch 60, p_succ: 0.3818281590938568
Epoch 70, p_succ: 0.3821461796760559
Epoch 80, p_succ: 0.3822503983974457
Training for p = 16
Epoch 0, p_succ: 0.0019072904251515865
Epoch 10, p_succ: 0.556391179561615
Epoch 20, p_succ: 0.6051459312438965
Epoch 30, p_succ: 0.6316158771514893
Epoch 40, p_succ: 0.6377452611923218
Epoch 50, p_succ: 0.6419916749000549
Epoch 60, p_succ: 0.6431066989898682
Epoch 70, p_succ: 0.6433513760566711
Epoch 80, p_succ: 0.643475353717804
